In [192]:
import torch
import numpy as np
import yfinance as yf
import statsmodels as sm
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet
from torch.utils.data import random_split
from statsmodels.tsa.stattools import adfuller
import pytorch_lightning as pl 
from arch import arch_model
from torch.utils.data import DataLoader

In [193]:
tickers = ['^GSPC', '^DJI', 'NQ=F', 'EURUSD=X', 'GC=F']
data = yf.download(tickers, start="2015-01-01", end="2025-01-01", group_by='ticker')

[*********************100%***********************]  5 of 5 completed


In [194]:
closing_price = pd.DataFrame()

In [195]:
for ticker in tickers:
    closing_price[ticker] = data[ticker]['Close']

In [196]:
closing_price

,^GSPC,^DJI,NQ=F,EURUSD=X,GC=F
Date,,,,,
2015-01-01,NaN,NaN,NaN,1.209863,NaN
2015-01-02,2058.199951,17832.990234,4214.25,1.208941,1186.000000
2015-01-05,2020.579956,17501.650391,4161.75,1.194643,1203.900024
2015-01-06,2002.609985,17371.640625,4102.25,1.193902,1219.300049
2015-01-07,2025.900024,17584.519531,4151.50,1.187536,1210.599976
...,...,...,...,...,...
2024-12-25,NaN,NaN,NaN,1.040258,NaN
2024-12-26,6037.589844,43325.800781,22008.00,1.039955,2638.800049
2024-12-27,5970.839844,42992.210938,21698.50,1.042318,2617.199951


In [197]:
closing_price.dropna(inplace=True)

In [198]:
log_returns = np.log(closing_price/closing_price.shift(-1))

In [199]:
log_returns.dropna(inplace=True)

In [200]:
log_returns

,^GSPC,^DJI,NQ=F,EURUSD=X,GC=F
Date,,,,,
2015-01-02,0.018447,0.018755,0.012536,0.011897,-0.014980
2015-01-05,0.008933,0.007456,0.014400,0.000621,-0.012711
2015-01-06,-0.011563,-0.012180,-0.011934,0.005346,0.007161
2015-01-07,-0.017730,-0.018221,-0.019264,0.003320,0.001819
2015-01-08,0.008439,0.009567,0.007471,0.003379,-0.006270
...,...,...,...,...,...
2024-12-23,-0.010982,-0.009050,-0.012574,0.002615,-0.002943
2024-12-24,0.000406,-0.000664,0.000931,0.000603,-0.007150
2024-12-26,0.011117,0.007729,0.014163,-0.002270,0.008219


In [201]:
def getVariance(windowsize):
    volatility = ((log_returns**2).rolling(windowsize).sum())
    return volatility

In [202]:
volatility = getVariance(5)

In [203]:
volatility

,^GSPC,^DJI,NQ=F,EURUSD=X,GC=F
Date,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN
2015-01-08,0.000939,0.000979,0.000934,0.000193,0.000480
...,...,...,...,...,...
2024-12-23,0.001187,0.000907,0.003024,0.000249,0.000539
2024-12-24,0.000291,0.000222,0.001668,0.000245,0.000582
2024-12-26,0.000414,0.000281,0.001844,0.000057,0.000362


In [204]:
data = []
for i, ticker in enumerate(tickers):
    temp_df = pd.DataFrame({
        "time_idx": np.arange(len(log_returns)),  
        "group": i,  
        "log_returns": log_returns[ticker].values,  
        "volatility": volatility[ticker].values,  
    })
    data.append(temp_df)


data = pd.concat(data)


In [205]:
data.fillna(0,inplace=True)

In [206]:
data.isnull().sum()

time_idx       0
group          0
log_returns    0
volatility     0
dtype: int64

In [207]:
data = data.reset_index(drop=True)

In [208]:
data['log_returns'] = data['log_returns'] * 100
data['volatility'] = data['volatility'] * 100

In [209]:
data

,time_idx,group,log_returns,volatility
0,0,0,1.844721,0.000000
1,1,0,0.893325,0.000000
2,2,0,-1.156274,0.000000
3,3,0,-1.773017,0.000000
4,4,0,0.843932,0.093938
...,...,...,...,...
12540,2504,4,-0.294324,0.053944
12541,2505,4,-0.714997,0.058161
12542,2506,4,0.821926,0.036202
12543,2507,4,0.425014,0.018457


In [210]:
data_len =data.time_idx.max()

In [211]:
unique_groups = data['group'].unique()

In [212]:
group_training_datasets = {}
group_validation_datasets = {}
group_testing_datasets = {}

In [213]:
for group  in unique_groups: 
    group_train_data = data[(data['time_idx']<=data_len *0.8) & (data['group'] ==group)]
    group_test_data = data[(data['time_idx'] > data_len*0.8) & (data['time_idx'] <= data_len*0.9) & (data['group'] == group)]
    group_val_data = data[(data['time_idx']>data_len *0.9) & (data['group'] ==group)]

    training_dataset = TimeSeriesDataSet(
        group_train_data,
        group_ids=["group"],  
        target="volatility",   
        time_idx="time_idx",  
        max_encoder_length=5,  
        max_prediction_length=1,  
        time_varying_known_reals=["log_returns"], 
        time_varying_unknown_reals=["volatility"],
        target_normalizer=None,
    )

    test_dataset = TimeSeriesDataSet(
        group_test_data,
        group_ids=["group"],  
        target="volatility",   
        time_idx="time_idx",  
        max_encoder_length=5,  
        max_prediction_length=1,  
        time_varying_known_reals=["log_returns"], 
        time_varying_unknown_reals=["volatility"],
        target_normalizer=None,
    )

    val_dataset = TimeSeriesDataSet(
        group_train_data,
        group_ids=["group"],  
        target="volatility",   
        time_idx="time_idx",  
        max_encoder_length=5,  
        max_prediction_length=1,  
        time_varying_known_reals=["log_returns"], 
        time_varying_unknown_reals=["volatility"],
        target_normalizer=None,
    )

    group_training_datasets[group] = training_dataset
    group_testing_datasets[group] = test_dataset
    group_validation_datasets[group] = val_dataset

In [214]:
group_training_datasets[3]

TimeSeriesDataSet[length=2002](
	time_idx='time_idx',
	target='volatility',
	group_ids=['group'],
	weight=None,
	max_encoder_length=5,
	min_encoder_length=5,
	min_prediction_idx=0,
	min_prediction_length=1,
	max_prediction_length=1,
	static_categoricals=[],
	static_reals=[],
	time_varying_known_categoricals=[],
	time_varying_known_reals=['log_returns'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['volatility'],
	variable_groups={},
	constant_fill_strategy={},
	allow_missing_timesteps=False,
	lags={},
	add_relative_time_idx=False,
	add_target_scales=False,
	add_encoder_length=False,
	target_normalizer=TorchNormalizer(method='identity', center=True, transformation=None, method_kwargs={}),
	categorical_encoders={'__group_id__group': NaNLabelEncoder(add_nan=False, warn=True)},
	scalers={'log_returns': StandardScaler()},
	randomize_length=None,
	predict_mode=False
)

In [215]:
summary_statistics = []

for group, group_data in data.groupby('group'):
    group_length = group_data['log_returns'].count()
    mean_volatility = group_data['log_returns'].mean()
    sd_volatility = group_data['log_returns'].std()

    adf_res = adfuller(group_data['log_returns'].dropna())
    adf_statistic = adf_res[0]
    adf_pvalue = adf_res[1]

    summary_statistics.append({
        'Ticker': tickers[group],
        'Length': group_length,
        'Mean': mean_volatility,
        'SD': sd_volatility,
        'ADF': adf_statistic,
        'p-value': adf_pvalue
    })

summary_statisticsdf = pd.DataFrame(summary_statistics)

In [216]:
summary_statisticsdf

,Ticker,Length,Mean,SD,ADF,p-value
0,^GSPC,2509,-0.041849,1.128400,-15.694171,1.421814e-28
1,^DJI,2509,-0.034655,1.110641,-15.728378,1.298412e-28
2,NQ=F,2509,-0.064439,1.382714,-16.349450,2.942271e-29
3,EURUSD=X,2509,0.005976,0.507394,-21.836893,0.000000e+00
4,GC=F,2509,-0.031730,0.925881,-51.323933,0.000000e+00


In [217]:
# ADF statistic << p-value -> time series are non-stationary

In [218]:
from torch.utils.data import DataLoader

# Словари для хранения DataLoader
group_train_dataloaders = {}
group_test_dataloaders = {}
group_val_dataloaders = {}

for group in unique_groups:
    # Создание DataLoader для тренировочного набора
    group_train_dataloaders[group] = group_training_datasets[group].to_dataloader(batch_size=32, shuffle=False)
    
    # Создание DataLoader для тестового набора
    group_test_dataloaders[group] = group_testing_datasets[group].to_dataloader(batch_size=32, shuffle=False)
    
    # Создание DataLoader для валидационного набора
    group_val_dataloaders[group] = group_validation_datasets[group].to_dataloader(batch_size=32, shuffle=False)

In [219]:
# and load the first batch
x, y = next(iter(group_train_dataloaders[0]))
print("x =", x)
print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

x = {'encoder_cat': tensor([], size=(32, 5, 0), dtype=torch.int64), 'encoder_cont': tensor([[[ 1.5704,  0.0000],
         [ 0.7739,  0.0000],
         [-0.9420,  0.0000],
         [-1.4584,  0.0000],
         [ 0.7326,  0.0939]],

        [[ 0.7739,  0.0000],
         [-0.9420,  0.0000],
         [-1.4584,  0.0000],
         [ 0.7326,  0.0939],
         [ 0.7064,  0.0665]],

        [[-0.9420,  0.0000],
         [-1.4584,  0.0000],
         [ 0.7326,  0.0939],
         [ 0.7064,  0.0665],
         [ 0.2422,  0.0592]],

        [[-1.4584,  0.0000],
         [ 0.7326,  0.0939],
         [ 0.7064,  0.0665],
         [ 0.2422,  0.0592],
         [ 0.5141,  0.0492]],

        [[ 0.7326,  0.0939],
         [ 0.7064,  0.0665],
         [ 0.2422,  0.0592],
         [ 0.5141,  0.0492],
         [ 0.8039,  0.0264]],

        [[ 0.7064,  0.0665],
         [ 0.2422,  0.0592],
         [ 0.5141,  0.0492],
         [ 0.8039,  0.0264],
         [-1.0904,  0.0371]],

        [[ 0.2422,  0.0592],
     

In [ ]:
from arch import arch_model

# Словарь для хранения обученных GARCH моделей
garch_models = {}

for group, dataloader in group_train_dataloaders.items():
    # Извлекаем данные для текущей группы
    for batch in dataloader:

        x, y = batch
        log_returns = x['encoder_cont'][:, :, 0].numpy()
        volatility = x['encoder_cont'][:, :, 1].numpy()
        log_returns = log_returns.flatten()
        volatility = volatility.flatten()
        # Обучаем GARCH(1,1) модель
        garch_model = arch_model(x=log_returns, y=volatility, vol='Garch', p=1, q=1, mean='Zero', rescale=False)
        garch_fitted = garch_model.fit(disp='off')
        
        # Сохраняем обученную модель
        garch_models[group] = garch_fitted
        
        # Выводим результаты
        print(f"GARCH(1,1) parameters for group {group}:")
        print(garch_fitted.params)
      # Обучаем только на первом батче для примера

GARCH(1,1) parameters for group 0:
omega       0.000044
alpha[1]    0.200000
beta[1]     0.780000
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000024
alpha[1]    0.200000
beta[1]     0.780000
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000015
alpha[1]    0.285571
beta[1]     0.683748
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000010
alpha[1]    0.196981
beta[1]     0.803019
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000152
alpha[1]    0.422879
beta[1]     0.562693
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000169
alpha[1]    0.211133
beta[1]     0.743529
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000039
alpha[1]    0.279937
beta[1]     0.687556
Name: params, dtype: float64
GARCH(1,1) parameters for group 0:
omega       0.000061
alpha[1]    0.418294
beta[1]     0.581706
Name: params,